# Full_Trip_Prediction

![Project Logo](https://github.com/peter-fayz/Full_Trip_Prediction/assets/138496693/54182e40-fbcd-4561-afc3-4fa860251431)

## Introduction

Full_Trip_Prediction is a machine learning project that aims to predict the percentage of orders that will be restocked using data exported from Shopify. The project utilizes historical orders data to train a predictive model that can provide insights into inventory management and restocking decisions. This will save extra shipping costs and save a lot of time to return products to stock and prioritize the required orders


## Import libraries

In [178]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
from dash import Dash,html,callback,dcc
from dash.dependencies import Input,Output

pd.set_option('display.max_columns', None)

### Read DataFrame

In [179]:
df = pd.read_csv('orders_export_1.csv',low_memory=False)

## Explore Data

In [180]:
df

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
0,#8374,NaN,pending,NaN,fulfilled,2023-07-19 19:35:45 +0300,no,EGP,335.00,40.0,0.0,375.00,NaN,40.0,الشحن,2023-07-19 19:33:38 +0300,1,نسختك الأفضل,165.00,NaN,BOOKJUICE-0274,True,True,fulfilled,يد محمود عوض,٤٣٨ ج شارع النرجس شقة ٢٠ الدور الخامس_ زهراء ا...,٤٣٨ ج شارع النرجس شقة ٢٠ الدور الخامس_ زهراء ا...,NaN,NaN,NaN,NaN,SU,EG,2.01E+09,يد محمود عوض,٤٣٨ ج شارع النرجس شقة ٢٠ الدور الخامس_ زهراء ا...,٤٣٨ ج شارع النرجس شقة ٢٠ الدور الخامس_ زهراء ا...,NaN,NaN,NaN,NaN,SU,EG,2010921620,NaN,NaN,NaN,Cash on Delivery (COD),NaN,0.0,دار عصير الكتب للنشر والتوزيع,5.140000e+12,NaN,Low,shopify_draft_order,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6th of October,6th of October,rePQ4rS6RAFECxincUmiNDflW,Due on receipt,NaN,rePQ4rS6RAFECxincUmiNDflW
1,#8374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-19 19:33:38 +0300,1,ضعي حدًا للتوتر قبل أن يضع حدًا لكِ,210.00,NaN,BOOKJUICE-0324,True,True,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,دار عصير الكتب للنشر والتوزيع,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#8373,nohayoussef85@yahoo.com,pending,NaN,fulfilled,2023-07-17 14:01:29 +0300,yes,EGP,72.25,40.0,0.0,112.25,NaN,0.0,الشحن,2023-07-16 23:50:14 +0300,1,ضي يونس,72.25,85.0,DAWN-0165,True,True,fulfilled,ف,"التجمع الخامس/ النرجس ٧ / شارع سيد درويش, الدو...",التجمع الخامس/ النرجس ٧ / شارع سيد درويش,الدور التاني شقة ٤,NaN,القاهرة الجديدة,11774,C,EG,0109 811 11,ف,"التجمع الخامس/ النرجس ٧ / شارع سيد درويش, الدو...",التجمع الخامس/ النرجس ٧ / شارع سيد درويش,الدور التاني شقة ٤,NaN,القاهرة الجديدة,11774,C,EG,0109 811 11,NaN,NaN,NaN,Cash on Delivery (COD),NaN,0.0,دار دون للنشر والتوزيع,5.130000e+12,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cairo,Cairo,rVhFbHBcld52McGyzzoURwsyd,NaN,NaN,rVhFbHBcld52McGyzzoURwsyd
3,#8372,Hagarsakr69@gmail.com,pending,NaN,fulfilled,2023-07-17 14:01:30 +0300,yes,EGP,445.70,40.0,0.0,485.70,NaN,0.0,الشحن,2023-07-16 23:06:38 +0300,1,أقدام وأحلام,50.05,65.0,RAHEK-0001,True,True,fulfilled,ر,المنطقة الثامنه - المجاوره الثالثه,المنطقة الثامنه - المجاوره الثالثه,NaN,NaN,الشروق,12345,C,EG,0106 071 74,ر,المنطقة الثامنه - المجاوره الثالثه,المنطقة الثامنه - المجاوره الثالثه,NaN,NaN,الشروق,12345,C,EG,0106 071 74,NaN,NaN,NaN,Cash on Delivery (COD),NaN,0.0,دار رحيق الكتب للنشر والتوزيع,5.130000e+12,NaN,Low,web,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cairo,Cairo,rW1yc0ksHqfTeNLRusULAM45u,NaN,NaN,rW1yc0ksHqfTeNLRusULAM45u
4,#8372,Hagarsakr69@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-16 23:06:38 +0300,1,حسين وأصدقاؤه,50.05,65.0,RAHEK-0002,True,True,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,دار رحيق الكتب للنشر والتوزيع,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,..

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51948 entries, 0 to 51947
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Name                         51948 non-null  object 
 1   Email                        25661 non-null  object 
 2   Financial Status             10618 non-null  object 
 3   Paid at                      2966 non-null   object 
 4   Fulfillment Status           10618 non-null  object 
 5   Fulfilled at                 9694 non-null   object 
 6   Accepts Marketing            10618 non-null  object 
 7   Currency                     10618 non-null  object 
 8   Subtotal                     10618 non-null  float64
 9   Shipping                     10618 non-null  float64
 10  Taxes                        10618 non-null  float64
 11  Total                        10618 non-null  float64
 12  Discount Code                343 non-null    object 
 13  Discount Amount 

In [182]:
df.describe().T#.sort_values('count')

,count,mean,std,min,25%,50%,75%,max
Subtotal,10618.0,3.364045e+02,4.192858e+02,0.000000e+00,1.184550e+02,2.223750e+02,4.034750e+02,9.709000e+03
Shipping,10618.0,4.678883e+01,1.135217e+02,0.000000e+00,3.200000e+01,3.500000e+01,4.000000e+01,3.291000e+03
Taxes,10618.0,5.981164e-02,1.437407e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.550000e+01
Total,10618.0,3.832456e+02,4.390627e+02,0.000000e+00,1.540000e+02,2.630000e+02,4.500000e+02,9.744000e+03
Discount Amount,10618.0,1.647559e+00,1.400416e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.395500e+02
Lineitem quantity,51948.0,1.023966e+00,4.144567e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+01
Lineitem price,51948.0,6.754224e+01,5.108695e+01,0.000000e+00,3.825000e+01,5.850000e+01,8.000000e+01,1.850000e+03
Lineitem compare at price,40757.0,7.689382e+01,5.800240e+01,0.000000e+00,4.200000e+01,6.500000e+01,9.000000e+01,2.800000e+03
Refunded Amount,10618.0,2.661047e-01,1.298474e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.421000e+02
Id,10618.0,4.296286e+12,5.755626e+11,3.180000e+12,3.900000e+12,4.320000e+12,4.920000e+12,5.140000e+12


In [183]:
df.describe(include='object').T.sort_values('count')

,count,unique,top,freq
Note Attributes,5,5,customer_id: my4zOImETZSLF4KDEUlztRdyHJn2,1
Billing Company,22,18,عمارات المروة الجديدة شارع احمد تيسير بجانب ا...,3
Shipping Company,27,24,بنك البركة مصر,2
Tax 1 Name,28,1,GST 10%,28
Discount Code,343,42,Custom discount,58
Notes,590,526,.,13
Tags,599,78,Pending,112
Cancelled at,978,977,2023-03-22 10:09:54 +0200,2
Payment Terms Name,2420,1,Due on receipt,2420
Payment Reference,2966,2966,rFvJ4ANUa0REKPTKPgIRwkDYV,1


In [184]:
df.duplicated().sum()

7

## Data Cleaning Stage

In [185]:
# Make column names lowercase for ease of handling

df.columns = df.columns.str.lower()

In [186]:
# Drop duplicates

df.drop_duplicates(inplace=True)

In [187]:
len(df.columns)

75

In [188]:
# save the prsantage of nan values in variable

nan_per = df.isna().sum()/len(df)*100
nan_per

name                        0.000000
email                      50.603569
financial status           79.557575
paid at                    94.289675
fulfillment status         79.557575
                             ...    
shipping province name     79.815560
payment id                 79.636511
payment terms name         95.340868
next payment due at       100.000000
payment references         79.636511
Length: 75, dtype: float64

In [189]:
nan_cols = nan_per[nan_per >= 100].index
print(len(nan_cols))
nan_cols # Show columns have 90% or more NaN values

11


Index(['tax 2 name', 'tax 2 value', 'tax 3 name', 'tax 3 value', 'tax 4 name',
       'tax 4 value', 'tax 5 name', 'tax 5 value', 'receipt number', 'duties',
       'next payment due at'],
      dtype='object')

In [190]:
df.drop(nan_cols,axis=1,inplace=True) #drop columns have 90% or more NaN values

In [191]:
no_var = df.nunique()[df.nunique()==1].index # columns with 0 variance 
no_var

Index(['currency', 'risk level', 'tax 1 name', 'payment terms name'], dtype='object')

In [192]:
df.drop(no_var,axis=1,inplace=True) #drop columns with 0 variance

In [193]:
len(df.columns)

60

In [194]:
def bad_Look_Investigator(df) :   # Showing the nunique, count and value counts for each column 
    for i in df.columns :
        print(f'{i}: Num of uniques {df[i].nunique()}, Count of values {df[i].count()}: \n')
        print(df[i].value_counts())
        print(100*'*')
        print(120*'*')

In [195]:
bad_Look_Investigator(df) #Investigation

name: Num of uniques 10618, Count of values 51941: 

name
BUZZ#3633    95
#7462        87
#7406        83
#5366        80
#5512        78
             ..
#7887         1
#7888         1
#7889         1
BUZZ#3425     1
#1656         1
Name: count, Length: 10618, dtype: int64
****************************************************************************************************
************************************************************************************************************************
email: Num of uniques 3645, Count of values 25657: 

email
osama.elzero@gmail.com         147
omarghaly@gmail.com            143
saadhwaida0@gmail.com          142
hoormohmed2021@gmail.com       122
adelbadr@hotmail.com           119
                              ... 
ayagaafer623@gmail.com           1
chaimabenali9898@gmail.com       1
asmaa19mohammed98@gmail.com      1
mr8115287@gmail.com              1
saleh_elgendy@hotmail.com        1
Name: count, Length: 3645, dtype: int64
*******************

In [196]:
#Select columns that I will not use in model

unwanted_cols = ['email','fulfillment status','taxes','discount amount','shipping method','lineitem name',
                 'lineitem sku','lineitem requires shipping','lineitem taxable','lineitem fulfillment status','billing name',
                 'billing street','billing address1','billing address2','billing company','billing city','billing phone',
                 'shipping name','shipping street','shipping address1','shipping address2','shipping company','shipping city',
                 'shipping zip','shipping phone','notes','note attributes','cancelled at','payment reference','refunded amount',
                 'id','tags','lineitem discount','tax 1 value','phone','billing province name','shipping province name',
                 'payment id','payment references',
                 
                 'discount code','billing zip','billing country','shipping country','subtotal','lineitem price',
                 'lineitem compare at price']

len(unwanted_cols)

46

In [197]:
'''

unwanted_cols = ['email','fulfillment status','currency','taxes','discount amount','shipping method',
             'lineitem name','lineitem sku','lineitem requires shipping','lineitem taxable','lineitem fulfillment status',
             'billing name','billing street','billing address1','billing address2','billing city','billing province',
             'billing country','billing phone','shipping name','shipping street','shipping address1','shipping address2',
             'shipping city','shipping province','shipping country','shipping phone','payment method','refunded amount',
             'id','risk level','lineitem discount','phone','shipping province name','payment id','payment references',
                'lineitem compare at price','subtotal','total']
len(unwanted_cols)'''

"\n\nunwanted_cols = ['email','fulfillment status','currency','taxes','discount amount','shipping method',\n             'lineitem name','lineitem sku','lineitem requires shipping','lineitem taxable','lineitem fulfillment status',\n             'billing name','billing street','billing address1','billing address2','billing city','billing province',\n             'billing country','billing phone','shipping name','shipping street','shipping address1','shipping address2',\n             'shipping city','shipping province','shipping country','shipping phone','payment method','refunded amount',\n             'id','risk level','lineitem discount','phone','shipping province name','payment id','payment references',\n                'lineitem compare at price','subtotal','total']\nlen(unwanted_cols)"

In [198]:
# Drop unwanted columns

df.drop(unwanted_cols,axis=1,inplace=True)

In [199]:
len(df.columns)

14

#### Cheack vendor colmn

In [200]:
df['vendor'].nunique(),df['vendor'].value_counts()

(134,
 vendor
 دار الشروق للنشر والتوزيع        5708
 دار الفاروق للنشر والتوزيع       5317
 الدار المصرية اللبنانية          3865
 دار عصير الكتب للنشر والتوزيع    3307
 دار دون للنشر والتوزيع           2853
                                  ... 
 المركز الثقافي العربي               1
 دار دال للنشر و التوزيع             1
 دار خطاب للنشر و التوزيع            1
 Mind Feed                           1
 دار الآداب                          1
 Name: count, Length: 134, dtype: int64)

In [201]:
# Find vendos that have appeared less than or equal 10 times

low_apper_vendors = df['vendor'].value_counts()[df['vendor'].value_counts()<=20].index

In [202]:
len(low_apper_vendors)

49

In [203]:
# Replace these vendors to be one vendor (others)

df['vendor'].replace(low_apper_vendors,'others',inplace=True)

In [204]:
# Fill nan values with the same vendo (others)

df['vendor'].fillna('others',inplace=True)

In [205]:
# Check again

df['vendor'].nunique()

86

### Let's check again

In [206]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51941 entries, 0 to 51947
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               51941 non-null  object 
 1   financial status   10618 non-null  object 
 2   paid at            2966 non-null   object 
 3   fulfilled at       9694 non-null   object 
 4   accepts marketing  10618 non-null  object 
 5   shipping           10618 non-null  float64
 6   total              10618 non-null  float64
 7   created at         51941 non-null  object 
 8   lineitem quantity  51941 non-null  int64  
 9   billing province   10429 non-null  object 
 10  shipping province  10484 non-null  object 
 11  payment method     10577 non-null  object 
 12  vendor             51941 non-null  object 
 13  source             10618 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 5.9+ MB


In [207]:
df.isna().sum()/len(df)*100

name                  0.000000
financial status     79.557575
paid at              94.289675
fulfilled at         81.336516
accepts marketing    79.557575
shipping             79.557575
total                79.557575
created at            0.000000
lineitem quantity     0.000000
billing province     79.921449
shipping province    79.815560
payment method       79.636511
vendor                0.000000
source               79.557575
dtype: float64

In [208]:
# Explore lineitem compare at price column

#len(df[df['lineitem compare at price'].isna()])

In [209]:
# NaN values in lineitem compare at price will make a bad effect in model 
# So I should fill NaNs with the same value of price (means no discount on this prouduct in this order)

#df['lineitem compare at price'].fillna(df['lineitem price'],inplace=True)

In [210]:
df.isna().sum()/len(df)*100

name                  0.000000
financial status     79.557575
paid at              94.289675
fulfilled at         81.336516
accepts marketing    79.557575
shipping             79.557575
total                79.557575
created at            0.000000
lineitem quantity     0.000000
billing province     79.921449
shipping province    79.815560
payment method       79.636511
vendor                0.000000
source               79.557575
dtype: float64

In [211]:
# reset index

df.reset_index(inplace=True,drop=True)

In [212]:
# Take another look on datafarame

df

,name,financial status,paid at,fulfilled at,accepts marketing,shipping,total,created at,lineitem quantity,billing province,shipping province,payment method,vendor,source
0,#8374,pending,NaN,2023-07-19 19:35:45 +0300,no,40.0,375.00,2023-07-19 19:33:38 +0300,1,SU,SU,Cash on Delivery (COD),دار عصير الكتب للنشر والتوزيع,shopify_draft_order
1,#8374,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-19 19:33:38 +0300,1,NaN,NaN,NaN,دار عصير الكتب للنشر والتوزيع,NaN
2,#8373,pending,NaN,2023-07-17 14:01:29 +0300,yes,40.0,112.25,2023-07-16 23:50:14 +0300,1,C,C,Cash on Delivery (COD),دار دون للنشر والتوزيع,web
3,#8372,pending,NaN,2023-07-17 14:01:30 +0300,yes,40.0,485.70,2023-07-16 23:06:38 +0300,1,C,C,Cash on Delivery (COD),دار رحيق الكتب للنشر والتوزيع,web
4,#8372,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-16 23:06:38 +0300,1,NaN,NaN,NaN,دار رحيق الكتب للنشر والتوزيع,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51936,BUZZ#1004,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-30 19:28:05 +0200,1,NaN,NaN,NaN,others,NaN
51937,BUZZ#1004,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-30 19:28:05 +0200,1,NaN,NaN,NaN,others,NaN
51938,BUZZ#1004,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-30 19:28:05 +0200,1,NaN,NaN,NaN,others,NaN
51939,BUZZ#1004,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-30 19:28:05 +0200,1,NaN,NaN,NaN,others,NaN


#### Now the dataframe is ready let's go to the next step

## Feature Engineering

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51941 entries, 0 to 51940
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               51941 non-null  object 
 1   financial status   10618 non-null  object 
 2   paid at            2966 non-null   object 
 3   fulfilled at       9694 non-null   object 
 4   accepts marketing  10618 non-null  object 
 5   shipping           10618 non-null  float64
 6   total              10618 non-null  float64
 7   created at         51941 non-null  object 
 8   lineitem quantity  51941 non-null  int64  
 9   billing province   10429 non-null  object 
 10  shipping province  10484 non-null  object 
 11  payment method     10577 non-null  object 
 12  vendor             51941 non-null  object 
 13  source             10618 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 5.5+ MB


## Group the data

In [214]:
# add dummies to df to group it

#new_df = pd.concat([df,pd.get_dummies(df['vendor'])],axis=1)
new_df = df.copy()
new_df.drop('vendor',axis=1,inplace=True)

In [215]:
'''from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

binary = encoder.fit_transform(new_df[['vendor']])

binary_vendors = pd.DataFrame(binary,columns=encoder.get_feature_names_out())

new_df = pd.concat([new_df,binary_vendors],axis=1)

new_df.drop('vendor',axis=1,inplace=True)'''

"from sklearn.preprocessing import OneHotEncoder\n\nencoder = OneHotEncoder(sparse_output=False)\n\nbinary = encoder.fit_transform(new_df[['vendor']])\n\nbinary_vendors = pd.DataFrame(binary,columns=encoder.get_feature_names_out())\n\nnew_df = pd.concat([new_df,binary_vendors],axis=1)\n\nnew_df.drop('vendor',axis=1,inplace=True)"

In [216]:
new_df

,name,financial status,paid at,fulfilled at,accepts marketing,shipping,total,created at,lineitem quantity,billing province,shipping province,payment method,source
0,#8374,pending,NaN,2023-07-19 19:35:45 +0300,no,40.0,375.00,2023-07-19 19:33:38 +0300,1,SU,SU,Cash on Delivery (COD),shopify_draft_order
1,#8374,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-19 19:33:38 +0300,1,NaN,NaN,NaN,NaN
2,#8373,pending,NaN,2023-07-17 14:01:29 +0300,yes,40.0,112.25,2023-07-16 23:50:14 +0300,1,C,C,Cash on Delivery (COD),web
3,#8372,pending,NaN,2023-07-17 14:01:30 +0300,yes,40.0,485.70,2023-07-16 23:06:38 +0300,1,C,C,Cash on Delivery (COD),web
4,#8372,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-16 23:06:38 +0300,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51936,BUZZ#1004,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-30 19:28:05 +0200,1,NaN,NaN,NaN,NaN
51937,BUZZ#1004,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-30 19:28:05 +0200,1,NaN,NaN,NaN,NaN
51938,BUZZ#1004,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-30 19:28:05 +0200,1,NaN,NaN,NaN,NaN
51939,BUZZ#1004,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-30 19:28:05 +0200,1,NaN,NaN,NaN,NaN


In [217]:
# Split the columns by dtype to deal with each one separately 

objesct_cols = new_df.dtypes[new_df.dtypes =='object'].index[1:] # The slicing to remove name column

num_cols = new_df.dtypes[new_df.dtypes !='object'].index

### Object columns

In [218]:
# Group object data using mode method

object_df = new_df.groupby('name',as_index=False)[objesct_cols].agg(pd.Series.mode)
object_df

,name,financial status,paid at,fulfilled at,accepts marketing,created at,billing province,shipping province,payment method,source
0,#1004,paid,2021-01-26 20:10:17 +0200,2021-01-26 19:31:33 +0200,no,2021-01-26 19:02:00 +0200,C,C,Cash on Delivery (COD),web
1,#1005,paid,2021-06-24 21:16:07 +0200,2021-01-27 16:41:58 +0200,no,2021-01-26 19:07:01 +0200,SHR,SHR,Cash on Delivery (COD),web
2,#1006,paid,2021-01-26 19:46:41 +0200,2021-01-28 15:06:22 +0200,no,2021-01-26 19:46:40 +0200,C,C,Paymob Accept - Card,web
3,#1007,paid,2021-06-24 21:16:07 +0200,2021-01-28 15:08:19 +0200,no,2021-01-26 19:48:17 +0200,C,C,Cash on Delivery (COD),web
4,#1008,paid,2021-06-24 21:16:07 +0200,2021-01-30 22:07:14 +0200,no,2021-01-26 20:14:22 +0200,DK,DK,Cash on Delivery (COD),web
...,...,...,...,...,...,...,...,...,...,...
10613,BUZZ#4272,pending,[],2023-07-19 14:16:14 +0300,no,2023-07-17 18:53:12 +0300,C,C,Cash on Delivery (COD),web
10614,BUZZ#4273,pending,[],2023-07-19 14:16:11 +0300,no,2023-07-18 19:45:11 +0300,GZ,GZ,manual,android
10615,BUZZ#4274,pending,[],2023-07-19 15:08:13 +0300,no,2023-07-19 14:51:17 +0300,SU,SU,Cash on Delivery (COD),web
10616,BUZZ#4275,voided,[],[],no,2023-07-19 16:57:51 +0300,[],[],Cash on Delivery (COD),web


In [219]:
# Cleaning financial status column (The Target 🎯)

object_df['financial status'].replace('partially_refunded','voided',inplace=True)
object_df['financial status'].replace('refunded','voided',inplace=True)


object_df['financial status'].replace('partially_paid','full_trip',inplace=True)
object_df['financial status'].replace('paid','full_trip',inplace=True)
object_df['financial status'].replace('pending','full_trip',inplace=True)

In [220]:
def paid_or_not(paid) : 
    try :
        if paid.isascii() :
            return 1
    except :
        return 0

In [221]:
object_df['paid at'] = object_df['paid at'].apply(paid_or_not)

In [222]:
# convert created at column to pandas datetime type 

object_df['created at'] = pd.to_datetime(object_df['created at'],utc=True)

# What I need from created at column month, day and hour

object_df['created month'] = object_df['created at'].dt.month
object_df['created day'] = object_df['created at'].dt.day
#object_df['created hour'] = object_df['created at'].dt.hour

# There is no need to created at column

object_df.drop('created at', axis=1, inplace=True)

In [223]:
# Function to handel numpy.ndarray in billing province name column

def fix_city(city) : 
    try :
        if city.isascii() :
            return city
        
    except :
        return np.nan
    

In [224]:
# Convert numpy.ndarray to numpy.nan

object_df['billing province'] = object_df['billing province'].apply(fix_city)

object_df['shipping province'] = object_df['shipping province'].apply(fix_city)




# Fill nans with mode in billing province name column

object_df['billing province'].fillna(object_df['billing province'].mode()[0],inplace=True)

object_df['shipping province'].fillna(object_df['shipping province'].mode()[0],inplace=True)

In [225]:
'''# convert fulfilled at column to pandas datetime type 

object_df['fulfilled at'] = pd.to_datetime(object_df['fulfilled at'].apply(fix_city).fillna(0),utc=True)

# What I need from created at column month, day and hour

object_df['fulfilled month'] = object_df['fulfilled at'].dt.month
object_df['fulfilled day'] = object_df['fulfilled at'].dt.day
#object_df['created hour'] = object_df['created at'].dt.hour

# There is no need to created at column

object_df.drop('fulfilled at', axis=1, inplace=True)'''

"# convert fulfilled at column to pandas datetime type \n\nobject_df['fulfilled at'] = pd.to_datetime(object_df['fulfilled at'].apply(fix_city).fillna(0),utc=True)\n\n# What I need from created at column month, day and hour\n\nobject_df['fulfilled month'] = object_df['fulfilled at'].dt.month\nobject_df['fulfilled day'] = object_df['fulfilled at'].dt.day\n#object_df['created hour'] = object_df['created at'].dt.hour\n\n# There is no need to created at column\n\nobject_df.drop('fulfilled at', axis=1, inplace=True)"

In [226]:
object_df.drop('fulfilled at',axis=1,inplace=True)

In [227]:
def payment_method(method) : 
    try :
        if method == "Cash on Delivery (COD)" :
            return method
        else :
            return 'others'

        
    except :
        return "Cash on Delivery (COD)"
        

In [228]:
object_df['payment method'].apply(payment_method)

C:\Users\LENONO\AppData\Local\Temp\ipykernel_10028\3050920208.py:3: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.



0        Cash on Delivery (COD)
1        Cash on Delivery (COD)
2                        others
3        Cash on Delivery (COD)
4        Cash on Delivery (COD)
                  ...          
10613    Cash on Delivery (COD)
10614                    others
10615    Cash on Delivery (COD)
10616    Cash on Delivery (COD)
10617                    others
Name: payment method, Length: 10618, dtype: object

In [229]:
# payment method

object_df['payment method'] = object_df['payment method'].apply(payment_method)


object_df['payment method'].fillna(object_df['payment method'].mode()[0],inplace=True)

C:\Users\LENONO\AppData\Local\Temp\ipykernel_10028\3050920208.py:3: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.



In [230]:
object_df['payment method']

0        Cash on Delivery (COD)
1        Cash on Delivery (COD)
2                        others
3        Cash on Delivery (COD)
4        Cash on Delivery (COD)
                  ...          
10613    Cash on Delivery (COD)
10614                    others
10615    Cash on Delivery (COD)
10616    Cash on Delivery (COD)
10617                    others
Name: payment method, Length: 10618, dtype: object

In [231]:
object_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10618 entries, 0 to 10617
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               10618 non-null  object
 1   financial status   10618 non-null  object
 2   paid at            10618 non-null  int64 
 3   accepts marketing  10618 non-null  object
 4   billing province   10618 non-null  object
 5   shipping province  10618 non-null  object
 6   payment method     10618 non-null  object
 7   source             10618 non-null  object
 8   created month      10618 non-null  int32 
 9   created day        10618 non-null  int32 
dtypes: int32(2), int64(1), object(7)
memory usage: 746.7+ KB


In [232]:
bad_Look_Investigator(object_df) #Investigation

name: Num of uniques 10618, Count of values 10618: 

name
#1004        1
#8092        1
#8094        1
#8095        1
#8096        1
            ..
#4548        1
#4549        1
#4550        1
#4551        1
BUZZ#4276    1
Name: count, Length: 10618, dtype: int64
****************************************************************************************************
************************************************************************************************************************
financial status: Num of uniques 2, Count of values 10618: 

financial status
full_trip    9662
voided        956
Name: count, dtype: int64
****************************************************************************************************
************************************************************************************************************************
paid at: Num of uniques 2, Count of values 10618: 

paid at
0    7652
1    2966
Name: count, dtype: int64
**********************************************

In [233]:
# Cleaning source column 

object_df['source'].replace('1830279','web',inplace=True)

In [234]:
# Cleaning billing province column

billing_cities = object_df['billing province'].value_counts()
billing_cities_out_of_egypt = billing_cities[billing_cities < 10].index

object_df['billing province'].replace(billing_cities_out_of_egypt,'out of egypt',inplace=True)

In [235]:


# Cleaning shipping province column

shipping_cities = object_df['shipping province'].value_counts()
shipping_cities_out_of_egypt = shipping_cities[shipping_cities < 10].index

object_df['shipping province'].replace(shipping_cities_out_of_egypt,'out of egypt',inplace=True)

In [236]:
# Cleaning source column

object_df['source'].replace('android','shopify_draft_order',inplace=True)

### Numerical columns

In [237]:
# Group numerical data using mode method

num_df = new_df.groupby('name',as_index=False)[num_cols].sum()
num_df

,name,shipping,total,lineitem quantity
0,#1004,32.0,213.50,3
1,#1005,44.0,81.40,1
2,#1006,32.0,363.93,3
3,#1007,32.0,300.40,5
4,#1008,44.0,364.71,4
...,...,...,...,...
10613,BUZZ#4272,40.0,88.00,1
10614,BUZZ#4273,40.0,125.00,1
10615,BUZZ#4274,40.0,142.00,1
10616,BUZZ#4275,1185.0,1410.00,1


In [238]:
num_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10618 entries, 0 to 10617
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               10618 non-null  object 
 1   shipping           10618 non-null  float64
 2   total              10618 non-null  float64
 3   lineitem quantity  10618 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 331.9+ KB


In [239]:
bad_Look_Investigator(num_df) #Investigation

name: Num of uniques 10618, Count of values 10618: 

name
#1004        1
#8092        1
#8094        1
#8095        1
#8096        1
            ..
#4548        1
#4549        1
#4550        1
#4551        1
BUZZ#4276    1
Name: count, Length: 10618, dtype: int64
****************************************************************************************************
************************************************************************************************************************
shipping: Num of uniques 87, Count of values 10618: 

shipping
32.0      4812
35.0      1903
44.0      1125
50.0       951
40.0       555
          ... 
440.0        1
850.0        1
1000.0       1
75.0         1
2335.0       1
Name: count, Length: 87, dtype: int64
****************************************************************************************************
************************************************************************************************************************
total: Num of uniques 4390

In [240]:
# Handel Outliers

'''for i in num_df.columns[1:3] :
    Q1 = np.percentile(num_df[i], 30)
    Q3 = np.percentile(num_df[i], 80)
    IQR = Q3 - Q1

    # Define the upper and lower bounds for outliers
    lower_bound = Q1 - 2.5 * IQR
    upper_bound = Q3 + 2.5 * IQR

    # Identify the outliers
    outlier_index = [i for i, x in enumerate(num_df[i]) if x < lower_bound or x > upper_bound]
    
    # Drp outliers using indexs
    num_df.drop(outlier_index,axis=0,inplace=True)
    num_df.reset_index(drop=True,inplace=True)
    
num_df.reset_index(inplace=True,drop=True)'''

'for i in num_df.columns[1:3] :\n    Q1 = np.percentile(num_df[i], 30)\n    Q3 = np.percentile(num_df[i], 80)\n    IQR = Q3 - Q1\n\n    # Define the upper and lower bounds for outliers\n    lower_bound = Q1 - 2.5 * IQR\n    upper_bound = Q3 + 2.5 * IQR\n\n    # Identify the outliers\n    outlier_index = [i for i, x in enumerate(num_df[i]) if x < lower_bound or x > upper_bound]\n    \n    # Drp outliers using indexs\n    num_df.drop(outlier_index,axis=0,inplace=True)\n    num_df.reset_index(drop=True,inplace=True)\n    \nnum_df.reset_index(inplace=True,drop=True)'

In [241]:
len(num_df)

10618

In [242]:
num_df.columns

Index(['name', 'shipping', 'total', 'lineitem quantity'], dtype='object')

In [243]:
for i in num_df.columns[1:] :
    fig = px.histogram(num_df[i])
    fig.show()

In [244]:
# Make a new dataframe to merge object and numerical dfs

new_df = pd.merge(object_df,num_df)
new_df

,name,financial status,paid at,accepts marketing,billing province,shipping province,payment method,source,created month,created day,shipping,total,lineitem quantity
0,#1004,full_trip,1,no,C,C,Cash on Delivery (COD),web,1,26,32.0,213.50,3
1,#1005,full_trip,1,no,SHR,SHR,Cash on Delivery (COD),web,1,26,44.0,81.40,1
2,#1006,full_trip,1,no,C,C,others,web,1,26,32.0,363.93,3
3,#1007,full_trip,1,no,C,C,Cash on Delivery (COD),web,1,26,32.0,300.40,5
4,#1008,full_trip,1,no,DK,DK,Cash on Delivery (COD),web,1,26,44.0,364.71,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10613,BUZZ#4272,full_trip,0,no,C,C,Cash on Delivery (COD),web,7,17,40.0,88.00,1
10614,BUZZ#4273,full_trip,0,no,GZ,GZ,others,shopify_draft_order,7,18,40.0,125.00,1
10615,BUZZ#4274,full_trip,0,no,SU,SU,Cash on Delivery (COD),web,7,19,40.0,142.00,1
10616,BUZZ#4275,voided,0,no,C,C,Cash on Delivery (COD),web,7,19,1185.0,1410.00,1


In [245]:
# Drop name after merge

new_df.drop('name',axis=1,inplace=True)

## Data Analysis & Visualizations

In [246]:
colors = {'background': '#F2F4F4','text': '#000000','plot_bgcolor' :'#F8F9F9'}

In [247]:
def update_layout (graph) :
    fig = graph.update_layout(paper_bgcolor=colors['background'],plot_bgcolor = colors['plot_bgcolor'],title_x=0.5)
    return fig

### df (main dataframe)

In [248]:
count_vendors = df.groupby('vendor')['lineitem quantity'].sum().sort_values(ascending=False)

vendors_hist = px.histogram(df,x= count_vendors.index , y= count_vendors,title='Count of books for each vendor'
                           ,color_discrete_sequence=px.colors.qualitative.Prism_r)
update_layout(vendors_hist)

In [249]:
vendors_sales = df.groupby('vendor')['total'].sum().sort_values(ascending = False)

vendors_sales_funnel = px.funnel(vendors_sales.head(15),color_discrete_sequence=px.colors.qualitative.Prism_r,
                                title='Total sales of each vendor')
update_layout(vendors_sales_funnel)


### new_df (ML)

In [250]:
financial_status = new_df['financial status'].value_counts()

financial_status_pie = px.pie(names=financial_status.index,values=financial_status,
                              title='The percentage between full trip and voided'
                              ,color_discrete_sequence=px.colors.qualitative.Set2_r)

update_layout(financial_status_pie)

In [251]:
markting = new_df['accepts marketing'].value_counts()

accepts_marketing_pie = px.pie(names=markting.index,values=markting,title='How many orders have accepted markting'
                              ,color_discrete_sequence=px.colors.qualitative.Set2_r)

update_layout(accepts_marketing_pie)

In [252]:
source = new_df['source'].value_counts()

sales_channels_pie = px.pie(names=source.index,values=source,title='Sales channels'
                           ,color_discrete_sequence=px.colors.qualitative.Set2_r)

update_layout(sales_channels_pie)

In [253]:
financial_status_df = pd.DataFrame()

financial_status_df['count'] = new_df['billing province'].value_counts()
financial_status_df['voided'] = new_df[new_df['financial status'] == 'voided']['billing province'].value_counts()

financial_status_df['persentage_of_voided'] = round(financial_status_df['voided']/financial_status_df['count']*100,2)


financial_status_df.reset_index(inplace=True)

fig = ff.create_table(financial_status_df)
fig.show()

In [254]:
countries_financial_status = px.histogram(new_df,x= 'billing province', color = 'financial status',barmode='group',
            title='Financial status for each country',color_discrete_sequence=px.colors.qualitative.Set2_r)

update_layout(countries_financial_status)

In [255]:
month_center = px.box(new_df,x='created month',y='created day',title='When are orders requested each month?'
                     ,color_discrete_sequence=px.colors.qualitative.Prism_r)

update_layout(month_center)

In [256]:
orders_per_month = px.histogram(new_df['created month'],title='Count of orders for each month'
                               ,color_discrete_sequence=px.colors.qualitative.Prism_r)

update_layout(orders_per_month)

In [257]:
countries_sales = new_df.groupby('billing province',as_index=False)['total'].sum()

line = px.line(countries_sales,'billing province','total',title='Countries total sales'
              ,color_discrete_sequence=px.colors.qualitative.Prism_r)

update_layout(line)

In [258]:
new_df.corr(numeric_only=True)

,paid at,created month,created day,shipping,total,lineitem quantity
paid at,1.000000,0.053284,-0.185154,0.013209,0.042658,0.074158
created month,0.053284,1.000000,0.022240,0.023248,-0.114225,-0.107338
created day,-0.185154,0.022240,1.000000,0.011180,-0.038568,-0.039025
shipping,0.013209,0.023248,0.011180,1.000000,0.299481,0.026881
total,0.042658,-0.114225,-0.038568,0.299481,1.000000,0.784127
lineitem quantity,0.074158,-0.107338,-0.039025,0.026881,0.784127,1.000000


In [259]:
px.imshow(new_df[new_df.columns[1:15]].corr(numeric_only=True),color_continuous_scale=px.colors.sequential.Brwnyl,
          title='The correlation between columns')

# Dashboard

In [260]:
app = Dash()



app.layout = html.Div([
    html.H1("Full Trip Prediction",style={'textAlign':'center'}),
    html.H3("Some analyzes and visualizations help us understand our business situation",style={'textAlign':'center'}),
    
    
    html.Div([
        
        dcc.Graph(id = 'vendors_hist',figure=vendors_hist),
        html.Br(),
        dcc.Graph(id = 'vendors_sales_funnel',figure=vendors_sales_funnel)
        
    ]),
    
    html.Br(),

    
    html.Div([
        dcc.Graph(id = 'financial_status_pie',figure=financial_status_pie
                ,style={'width': '33%', 'display': 'inline-block'}),
    
        dcc.Graph(id = 'sales_channels_pie',figure=sales_channels_pie
                ,style={'width': '37%', 'display': 'inline-block'}),

        dcc.Graph(id = 'accepts_marketing_pie',figure=accepts_marketing_pie
                ,style={'width': '30%', 'display': 'inline-block'})]),
    
    html.Br(),
    
    html.Div([
        dcc.Graph(id = 'countries_financial_status' , figure=countries_financial_status),
        dcc.Graph(id = 'fig' ,figure=fig)
    ]),
    
    html.Br(),
    
    html.Div([
        dcc.Graph(id = 'month_center' , figure=month_center
                 ,style={'width': '50%', 'display': 'inline-block'}),
        dcc.Graph(id = 'orders_per_month' ,figure=orders_per_month
                 ,style={'width': '50%', 'display': 'inline-block'})
    ]),
    
    html.Br(),
    
    html.Div([
        dcc.Graph(id = 'line',figure=line)
    ])
    
],style={'backgroundColor': colors['background']})
    

    
    
if __name__ == '__main__':
    app.run_server(debug=False, port=8057)

## Preprocessing

In [261]:
# Prepare data to preprocessing

x = new_df.drop('financial status',axis=1)
y = new_df['financial status']

### Encoding

In [262]:
# See the nunique for each column

more_than_3 = []
less_than_3= []

for i in x.select_dtypes(include='object').columns :
    print(f'Column Name: {i} , num of unique :{x[i].nunique()}')
    if x[i].nunique() > 3 :
        more_than_3.append(i)
    elif x[i].nunique() < 3 :
        less_than_3.append(i)

Column Name: accepts marketing , num of unique :2
Column Name: billing province , num of unique :27
Column Name: shipping province , num of unique :27
Column Name: payment method , num of unique :2
Column Name: source , num of unique :2


In [263]:
more_than_3

['billing province', 'shipping province']

In [264]:
less_than_3

['accepts marketing', 'payment method', 'source']

In [265]:
# Select object columns

object_cols = x.select_dtypes(include='object').columns

In [266]:
from category_encoders import BinaryEncoder  # OneHotEncoder

one1 = BinaryEncoder(drop_invariant=True)

encoded_cols = one1.fit_transform(x[object_cols])

one1_df = pd.DataFrame(encoded_cols,columns=one1.get_feature_names_out())

In [267]:
one_hot_df = pd.concat([one1_df,x.drop(object_cols,axis=1)],axis=1)

In [268]:
one_hot_df

,accepts marketing_0,accepts marketing_1,billing province_0,billing province_1,billing province_2,billing province_3,billing province_4,shipping province_0,shipping province_1,shipping province_2,shipping province_3,shipping province_4,payment method_0,payment method_1,source_0,source_1,paid at,created month,created day,shipping,total,lineitem quantity
0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,1,1,26,32.0,213.50,3
1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,26,44.0,81.40,1
2,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,1,26,32.0,363.93,3
3,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,1,1,26,32.0,300.40,5
4,0,1,0,0,0,1,1,0,0,0,1,1,0,1,0,1,1,1,26,44.0,364.71,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10613,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,7,17,40.0,88.00,1
10614,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,7,18,40.0,125.00,1
10615,0,1,0,1,1,1,0,0,1,1,1,0,0,1,0,1,0,7,19,40.0,142.00,1
10616,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,7,19,1185.0,1410.00,1


### Train test split

In [269]:
from  sklearn.model_selection import train_test_split

xtrain, xtest, ytrain,ytest=train_test_split(one_hot_df, y , test_size=0.2, random_state=42, stratify=y)

In [270]:
xtrain.shape, xtest.shape, ytrain.shape,ytest.shape

((8494, 22), (2124, 22), (8494,), (2124,))

### Scaling

In [271]:
from sklearn.preprocessing import MinMaxScaler # StandardScaler, Normalizer

std = MinMaxScaler()

xtrain_std = std.fit_transform(xtrain)
xtest_std = std.transform(xtest)

In [272]:
xtrain_std = pd.DataFrame(xtrain_std,columns=std.get_feature_names_out())

xtest_std = pd.DataFrame(xtest_std,columns=std.get_feature_names_out())

In [273]:
xtrain_std

,accepts marketing_0,accepts marketing_1,billing province_0,billing province_1,billing province_2,billing province_3,billing province_4,shipping province_0,shipping province_1,shipping province_2,shipping province_3,shipping province_4,payment method_0,payment method_1,source_0,source_1,paid at,created month,created day,shipping,total,lineitem quantity
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.727273,0.333333,0.010635,0.016061,0.010638
1,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.272727,0.100000,0.015193,0.048081,0.021277
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.090909,0.066667,0.009723,0.055475,0.042553
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.090909,0.166667,0.010635,0.211956,0.148936
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.000000,0.900000,0.009723,0.032225,0.042553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.090909,0.133333,0.012154,0.073235,0.117021
8490,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.636364,0.966667,0.012154,0.084924,0.063830
8491,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.090909,0.100000,0.009723,0.017806,0.021277
8492,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.545455,0.066667,0.009723,0.047568,0.074468


In [274]:
xtrain_std.shape

(8494, 22)

### Oversampling

In [275]:
ytrain.value_counts()/len(ytrain)*100

financial status
full_trip    90.993643
voided        9.006357
Name: count, dtype: float64

In [276]:
#from imblearn.under_sampling import RandomUnderSampler

from imblearn.over_sampling import RandomOverSampler #SMOTE  

smo = RandomOverSampler(random_state=52,sampling_strategy=.5) 

xtrain_std_som , ytrain_smo = smo.fit_resample(xtrain_std , ytrain)

In [277]:
xtrain_std_som = pd.DataFrame(xtrain_std_som,columns=smo.get_feature_names_out())

In [278]:
ytrain_smo.value_counts()/len(ytrain)*100

financial status
full_trip    90.993643
voided       45.490935
Name: count, dtype: float64

In [279]:
xtrain_std_som.shape , ytrain_smo.shape

((11593, 22), (11593,))

### Feature selection

In [280]:
from sklearn.ensemble import RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import classification_report

In [104]:
# Feature Selection on scaled data

sfs_std = SFS(RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=52), 
           k_features=10, 
           forward=True, 
           floating=False, 
           verbose=2, 
           scoring='accuracy',
           cv=3,
           n_jobs=-1
         )

sfs_std.fit(xtrain_std,ytrain)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done  19 out of  22 | elapsed:    9.0s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    9.6s finished

[2023-08-08 09:21:58] Features: 1/10 -- score: 0.911937697445753[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:    6.7s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    7.1s finished

[2023-08-08 09:22:06] Features: 2/10 -- score: 0.9168822396776074[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:    4.7s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    6.3s finished

[2023-08-08 09:22:12] Features: 3/10 -- score: 0.9171177693028688[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of  19 | elapsed:    4.6s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  19 out of

SequentialFeatureSelector(cv=3,
                          estimator=RandomForestClassifier(n_jobs=-1,
                                                           random_state=52),
                          k_features=(10, 10), n_jobs=-1, scoring='accuracy',
                          verbose=2)

In [105]:
sfs_std.subsets_

{1: {'feature_idx': (19,),
  'cv_scores': array([0.91313559, 0.91098552, 0.91169198]),
  'avg_score': 0.911937697445753,
  'feature_names': ('shipping',)},
 2: {'feature_idx': (16, 19),
  'cv_scores': array([0.91913842, 0.91557753, 0.91593077]),
  'avg_score': 0.9168822396776074,
  'feature_names': ('paid at', 'shipping')},
 3: {'feature_idx': (2, 16, 19),
  'cv_scores': array([0.91878531, 0.916284  , 0.916284  ]),
  'avg_score': 0.9171177693028688,
  'feature_names': ('billing province_0', 'paid at', 'shipping')},
 4: {'feature_idx': (2, 14, 16, 19),
  'cv_scores': array([0.91913842, 0.91699046, 0.91593077]),
  'avg_score': 0.9173532157755705,
  'feature_names': ('billing province_0', 'source_0', 'paid at', 'shipping')},
 5: {'feature_idx': (2, 3, 14, 16, 19),
  'cv_scores': array([0.91949153, 0.91699046, 0.91593077]),
  'avg_score': 0.9174709182237816,
  'feature_names': ('billing province_0',
   'billing province_1',
   'source_0',
   'paid at',
   'shipping')},
 6: {'feature_idx': 

In [106]:
# Feature Selection on scaled over sampled data

sfs_smo = SFS(RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=52), 
           k_features=10, 
           forward=True, 
           floating=False, 
           verbose=2, 
           scoring='accuracy',
           cv=3,
           n_jobs=-1 
         )

sfs_smo.fit(xtrain_std_som,ytrain_smo)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done  19 out of  22 | elapsed:    6.2s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    7.2s finished

[2023-08-08 09:23:01] Features: 1/10 -- score: 0.7796946102812573[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:   25.1s remaining:    5.8s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   26.2s finished

[2023-08-08 09:23:27] Features: 2/10 -- score: 0.8906242606535085[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:   16.1s remaining:    4.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   18.7s finished

[2023-08-08 09:23:46] Features: 3/10 -- score: 0.9154666375615248[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of  19 | elapsed:   16.7s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done  19 out o

SequentialFeatureSelector(cv=3,
                          estimator=RandomForestClassifier(n_jobs=-1,
                                                           random_state=52),
                          k_features=(10, 10), n_jobs=-1, scoring='accuracy',
                          verbose=2)

In [107]:
sfs_smo.subsets_

{1: {'feature_idx': (20,),
  'cv_scores': array([0.78007762, 0.78312629, 0.77587992]),
  'avg_score': 0.7796946102812573,
  'feature_names': ('total',)},
 2: {'feature_idx': (18, 20),
  'cv_scores': array([0.8835705 , 0.90113872, 0.88716356]),
  'avg_score': 0.8906242606535085,
  'feature_names': ('created day', 'total')},
 3: {'feature_idx': (17, 18, 20),
  'cv_scores': array([0.91073739, 0.91770186, 0.91796066]),
  'avg_score': 0.9154666375615248,
  'feature_names': ('created month', 'created day', 'total')},
 4: {'feature_idx': (17, 18, 20, 21),
  'cv_scores': array([0.93247089, 0.9371118 , 0.93012422]),
  'avg_score': 0.9332356391569508,
  'feature_names': ('created month',
   'created day',
   'total',
   'lineitem quantity')},
 5: {'feature_idx': (4, 17, 18, 20, 21),
  'cv_scores': array([0.94049159, 0.94668737, 0.94202899]),
  'avg_score': 0.9430693157702551,
  'feature_names': ('billing province_2',
   'created month',
   'created day',
   'total',
   'lineitem quantity')},
 6:

In [108]:
selected_columns = list(sfs_std.subsets_[10]['feature_names'])
selected_columns

['billing province_0',
 'billing province_1',
 'billing province_2',
 'shipping province_0',
 'shipping province_1',
 'shipping province_2',
 'source_0',
 'source_1',
 'paid at',
 'shipping']

In [109]:
random_fors_std = RandomForestClassifier(n_estimators=200,random_state=52,n_jobs=-1)
random_fors_std.fit(xtrain_std_som[selected_columns],ytrain_smo)
y_pred_std = random_fors_std.predict(xtest_std[selected_columns])
print(f'Classification Report score is : \n {classification_report(y_pred_std, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.90      0.92      0.91      1879
      voided       0.26      0.20      0.23       245

    accuracy                           0.84      2124
   macro avg       0.58      0.56      0.57      2124
weighted avg       0.83      0.84      0.83      2124



## Modeling

### RandomForestClassifier

In [110]:
random_fors_std = RandomForestClassifier(n_estimators=100,random_state=52,n_jobs=-1)
random_fors_std.fit(xtrain_std,ytrain)
y_pred_std = random_fors_std.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_std, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.98      0.92      0.95      2060
      voided       0.15      0.44      0.22        64

    accuracy                           0.91      2124
   macro avg       0.56      0.68      0.58      2124
weighted avg       0.96      0.91      0.93      2124



In [111]:
random_fors_smo = RandomForestClassifier(n_estimators=100 ,random_state=52)
random_fors_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = random_fors_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.97      0.92      0.94      2019
      voided       0.20      0.36      0.26       105

    accuracy                           0.90      2124
   macro avg       0.58      0.64      0.60      2124
weighted avg       0.93      0.90      0.91      2124



### Naive Bayes

In [112]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

In [113]:
nb_smo = GaussianNB()
nb_smo.fit(xtrain,ytrain)
y_pred_smo = nb_smo.predict(xtest)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.87      0.93      0.90      1821
      voided       0.30      0.19      0.23       303

    accuracy                           0.82      2124
   macro avg       0.59      0.56      0.56      2124
weighted avg       0.79      0.82      0.80      2124



In [114]:
nb_std = GaussianNB(var_smoothing=.054)
nb_std.fit(xtrain_std,ytrain)
y_pred_std = nb_std.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_std, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.87      0.93      0.90      1814
      voided       0.29      0.18      0.22       310

    accuracy                           0.82      2124
   macro avg       0.58      0.55      0.56      2124
weighted avg       0.78      0.82      0.80      2124



In [115]:
nb_smo = GaussianNB(var_smoothing=.8)
nb_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = nb_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.83      0.93      0.88      1728
      voided       0.34      0.16      0.22       396

    accuracy                           0.78      2124
   macro avg       0.58      0.55      0.55      2124
weighted avg       0.74      0.78      0.75      2124



In [116]:
# GridSearchCV

nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=250)}

gs_NB = GridSearchCV(estimator=nb_classifier, param_grid=params_NB, cv=5, verbose=1, scoring='accuracy')
gs_NB.fit(xtrain_std,ytrain)

nb_best_par = gs_NB.best_params_['var_smoothing']

gs_NB.best_params_

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


{'var_smoothing': 0.6069215106410442}

In [117]:
nb_std = GaussianNB(var_smoothing=nb_best_par)
nb_std.fit(xtrain_std,ytrain)
y_pred_std = nb_std.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_std, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       1.00      0.91      0.95      2124
      voided       0.00      0.00      0.00         0

    accuracy                           0.91      2124
   macro avg       0.50      0.46      0.48      2124
weighted avg       1.00      0.91      0.95      2124



d:\EpsilonAI\Courses\Data Science\Machine Learning\Final Project\env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

d:\EpsilonAI\Courses\Data Science\Machine Learning\Final Project\env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

d:\EpsilonAI\Courses\Data Science\Machine Learning\Final Project\env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [118]:
# GridSearchCV

nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,9, num=250)}

gs_NB = GridSearchCV(estimator=nb_classifier, param_grid=params_NB, cv=5, verbose=1, scoring='accuracy')
gs_NB.fit(xtrain_std_som,ytrain_smo)

nb_best_par = gs_NB.best_params_['var_smoothing']

gs_NB.best_params_

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


{'var_smoothing': 1.1811067639400605}

In [119]:
nb_smo = GaussianNB(var_smoothing=nb_best_par) # 2.4620924014946257
nb_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = nb_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.94      0.92      0.93      1974
      voided       0.18      0.23      0.21       150

    accuracy                           0.87      2124
   macro avg       0.56      0.58      0.57      2124
weighted avg       0.89      0.87      0.88      2124



In [120]:
nb_smo = GaussianNB(var_smoothing=.7) 
nb_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = nb_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.82      0.93      0.87      1698
      voided       0.38      0.17      0.24       426

    accuracy                           0.78      2124
   macro avg       0.60      0.55      0.55      2124
weighted avg       0.73      0.78      0.74      2124



### DecisionTreeClassifier

In [121]:
from sklearn.tree import DecisionTreeClassifier

In [122]:
nb_std = DecisionTreeClassifier(random_state=52)
nb_std.fit(xtrain_std,ytrain)
y_pred_std = nb_std.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_std, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.91      0.92      0.92      1906
      voided       0.24      0.21      0.22       218

    accuracy                           0.85      2124
   macro avg       0.58      0.57      0.57      2124
weighted avg       0.84      0.85      0.85      2124



In [123]:
nb_smo = DecisionTreeClassifier(random_state=80)
nb_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = nb_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.92      0.92      0.92      1916
      voided       0.24      0.22      0.23       208

    accuracy                           0.86      2124
   macro avg       0.58      0.57      0.58      2124
weighted avg       0.85      0.86      0.85      2124



In [124]:
# GridSearchCV

# define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10]
}

# create a decision tree classifier
dtc = DecisionTreeClassifier()

# create a grid search object
grid_search = GridSearchCV(dtc, param_grid)

# fit the grid search object to the data
grid_search.fit(xtrain, ytrain)

# print the best parameters
param = grid_search.best_params_
param

{'criterion': 'entropy',
 'max_depth': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [125]:
dt = DecisionTreeClassifier(
    criterion=param['criterion'],
    max_depth=param['max_depth'],
    min_samples_leaf=param['min_samples_leaf'],
    min_samples_split=param['min_samples_split']
)

dt.fit(xtrain,ytrain)
y_pred_smo = dt.predict(xtest)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       1.00      0.91      0.95      2110
      voided       0.06      0.79      0.11        14

    accuracy                           0.91      2124
   macro avg       0.53      0.85      0.53      2124
weighted avg       0.99      0.91      0.95      2124



In [126]:
# GridSearchCV

# define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10]
}

# create a decision tree classifier
dtc = DecisionTreeClassifier()

# create a grid search object
grid_search = GridSearchCV(dtc, param_grid)

# fit the grid search object to the data
grid_search.fit(xtrain_std, ytrain)

# print the best parameters
param = grid_search.best_params_
param

{'criterion': 'entropy',
 'max_depth': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [127]:
dt_std = DecisionTreeClassifier(
    criterion=param['criterion'],
    max_depth=param['max_depth'],
    min_samples_leaf=param['min_samples_leaf'],
    min_samples_split=param['min_samples_split']
)

dt_std.fit(xtrain_std,ytrain)
y_pred_smo = dt_std.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       1.00      0.91      0.95      2110
      voided       0.06      0.79      0.11        14

    accuracy                           0.91      2124
   macro avg       0.53      0.85      0.53      2124
weighted avg       0.99      0.91      0.95      2124



In [128]:
# GridSearchCV

# define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10]
}

# create a decision tree classifier
dtc = DecisionTreeClassifier()

# create a grid search object
grid_search = GridSearchCV(dtc, param_grid)

# fit the grid search object to the data
grid_search.fit(xtrain_std_som, ytrain_smo)

# print the best parameters
param = grid_search.best_params_
param

{'criterion': 'gini',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [129]:
dt_smo = DecisionTreeClassifier(      #{'criterion': 'entropy','max_depth': None,'min_samples_leaf': 1,'min_samples_split': 2}
    criterion=param['criterion'],
    max_depth=param['max_depth'],
    min_samples_leaf=param['min_samples_leaf'],
    min_samples_split=param['min_samples_split']
)

dt_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = dt_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.92      0.93      0.92      1914
      voided       0.26      0.23      0.24       210

    accuracy                           0.86      2124
   macro avg       0.59      0.58      0.58      2124
weighted avg       0.85      0.86      0.85      2124



### SVM

In [130]:
from sklearn.svm import SVC

In [174]:
nb_std = SVC(random_state=52)
nb_std.fit(xtrain_std,ytrain)
y_pred_std = nb_std.predict(xtest_std)
print(f'Classification Report score is : \n{classification_report(y_pred_std, ytest)}')

Classification Report score is : 
              precision    recall  f1-score   support

           0       0.03      1.00      0.06         6
           1       1.00      0.91      0.95      2118

    accuracy                           0.91      2124
   macro avg       0.52      0.96      0.51      2124
weighted avg       1.00      0.91      0.95      2124



In [281]:
nb_smo = SVC(random_state=52)
nb_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = nb_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.91      0.92      0.92      1903
      voided       0.24      0.20      0.22       221

    accuracy                           0.85      2124
   macro avg       0.57      0.56      0.57      2124
weighted avg       0.84      0.85      0.84      2124



### LogisticRegression

In [133]:
from sklearn.linear_model import LogisticRegression

In [134]:
nb_std = LogisticRegression(random_state=52,max_iter=500,C=20)
nb_std.fit(xtrain_std,ytrain)
y_pred_std = nb_std.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_std, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       1.00      0.91      0.95      2109
      voided       0.06      0.73      0.11        15

    accuracy                           0.91      2124
   macro avg       0.53      0.82      0.53      2124
weighted avg       0.99      0.91      0.95      2124



In [135]:
nb_smo = LogisticRegression(random_state=52,max_iter=200,C=20)
nb_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = nb_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.92      0.93      0.93      1909
      voided       0.32      0.28      0.30       215

    accuracy                           0.87      2124
   macro avg       0.62      0.61      0.61      2124
weighted avg       0.86      0.87      0.86      2124



### KNeighborsClassifier

In [136]:
from sklearn.neighbors import KNeighborsClassifier

In [137]:
knn_std = KNeighborsClassifier(n_neighbors=1)
knn_std.fit(xtrain_std,ytrain)
y_pred_std = knn_std.predict(np.ascontiguousarray(xtest_std))
print(f'Classification Report score is : \n {classification_report(y_pred_std, ytest)}')

d:\EpsilonAI\Courses\Data Science\Machine Learning\Final Project\env\lib\site-packages\sklearn\base.py:464: UserWarning:

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names



Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.92      0.93      0.92      1928
      voided       0.25      0.24      0.24       196

    accuracy                           0.86      2124
   macro avg       0.58      0.58      0.58      2124
weighted avg       0.86      0.86      0.86      2124



In [138]:
knn_smo = KNeighborsClassifier(n_neighbors=2)
knn_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = knn_smo.predict(np.ascontiguousarray(xtest_std))
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.92      0.92      0.92      1938
      voided       0.21      0.22      0.22       186

    accuracy                           0.86      2124
   macro avg       0.57      0.57      0.57      2124
weighted avg       0.86      0.86      0.86      2124



d:\EpsilonAI\Courses\Data Science\Machine Learning\Final Project\env\lib\site-packages\sklearn\base.py:464: UserWarning:

X does not have valid feature names, but KNeighborsClassifier was fitted with feature names



### GradientBoostingClassifier

In [139]:
from sklearn.ensemble import GradientBoostingClassifier

In [140]:
'''
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3]
             }
'''

"\nparam_grid = {\n    'n_estimators': [100, 200, 300],\n    'learning_rate': [0.1, 0.05, 0.01],\n    'max_depth': [3, 4, 5],\n    'min_samples_split': [2, 4, 6],\n    'min_samples_leaf': [1, 2, 3]\n             }\n"

In [141]:
knn_smo = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.3,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1
)

knn_smo.fit(xtrain_std,ytrain)
y_pred_smo = knn_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.99      0.92      0.95      2085
      voided       0.12      0.56      0.19        39

    accuracy                           0.91      2124
   macro avg       0.55      0.74      0.57      2124
weighted avg       0.98      0.91      0.94      2124



In [142]:
knn_smo = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.008,
    max_depth=3
)

knn_smo.fit(xtrain_std_som,ytrain_smo)
y_pred_smo = knn_smo.predict(xtest_std)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.95      0.94      0.94      1962
      voided       0.35      0.41      0.37       162

    accuracy                           0.90      2124
   macro avg       0.65      0.67      0.66      2124
weighted avg       0.90      0.90      0.90      2124



In [143]:
knn_smo = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.5,
    max_depth=10
)

knn_smo.fit(xtrain,ytrain)
y_pred_smo = knn_smo.predict(xtest)
print(f'Classification Report score is : \n {classification_report(y_pred_smo, ytest)}')

Classification Report score is : 
               precision    recall  f1-score   support

   full_trip       0.98      0.92      0.95      2047
      voided       0.18      0.44      0.25        77

    accuracy                           0.91      2124
   macro avg       0.58      0.68      0.60      2124
weighted avg       0.95      0.91      0.92      2124



### xgboost

In [144]:
import xgboost as xgb

In [145]:
xg_ytrain = ytrain.replace({'full_trip' : 1,'voided':0})
xg_ytest = ytest.replace({'full_trip' : 1,'voided':0})
xg_ytrain_smo = ytrain_smo.replace({'full_trip' : 1,'voided':0})

In [146]:
xg_ytrain.shape , xg_ytest.shape, xg_ytrain_smo.shape

((8494,), (2124,), (11593,))

In [147]:
xg_std = xgb.XGBClassifier()

xg_std.fit(xtrain_std,xg_ytrain)

y_pred_std = xg_std.predict(xtest_std)

print(classification_report(y_pred_std,xg_ytest))

              precision    recall  f1-score   support

           0       0.18      0.58      0.27        59
           1       0.99      0.92      0.95      2065

    accuracy                           0.91      2124
   macro avg       0.58      0.75      0.61      2124
weighted avg       0.96      0.91      0.94      2124



In [148]:
xg_std = xgb.XGBClassifier()

xg_std.fit(xtrain_std,xg_ytrain)

y_pred_std = xg_std.predict(xtest_std)

print(classification_report(y_pred_std,xg_ytest))

              precision    recall  f1-score   support

           0       0.18      0.58      0.27        59
           1       0.99      0.92      0.95      2065

    accuracy                           0.91      2124
   macro avg       0.58      0.75      0.61      2124
weighted avg       0.96      0.91      0.94      2124



In [149]:
xg_smo = xgb.XGBClassifier()

xg_smo.fit(xtrain_std_som,xg_ytrain_smo)

y_pred_smo = xg_smo.predict(xtest_std)

print(classification_report(y_pred_smo,xg_ytest))

              precision    recall  f1-score   support

           0       0.35      0.34      0.34       198
           1       0.93      0.94      0.93      1926

    accuracy                           0.88      2124
   macro avg       0.64      0.64      0.64      2124
weighted avg       0.88      0.88      0.88      2124



In [150]:
# GridSearchCV

# define the model
model = xgb.XGBClassifier()

# define the search space
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [ 4, 6, 8, 10, 12],
    'learning_rate': [0.5, 0.1,1.5, 0.2, 0.3]
}

# define the search
search = GridSearchCV(model, param_grid, scoring='accuracy', cv=5,n_jobs=-1)

# assuming X_train and y_train are your train data and labels
search.fit(xtrain,xg_ytrain)

# print the best hyperparameters
print(search.best_params_)


{'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 50}


In [151]:
xg_std = xgb.XGBClassifier(**search.best_params_)

xg_std.fit(xtrain,xg_ytrain)

y_pred_std = xg_std.predict(xtest)

print(classification_report(y_pred_std,xg_ytest))

              precision    recall  f1-score   support

           0       0.14      0.65      0.23        40
           1       0.99      0.92      0.96      2084

    accuracy                           0.92      2124
   macro avg       0.56      0.79      0.59      2124
weighted avg       0.98      0.92      0.94      2124



In [152]:
# GridSearchCV

# define the model
model = xgb.XGBClassifier()

# define the search space
param_grid = {
    'n_estimators': [150, 200, 250],
    'max_depth': [4, 6, 8, 10],
    'learning_rate': [0.05, 0.1, 0.25, 0.8]
}

# define the search
search = GridSearchCV(model, param_grid, scoring='accuracy', cv=5,n_jobs=-1)

# assuming X_train and y_train are your train data and labels
search.fit(xtrain_std_som,xg_ytrain_smo)

# print the best hyperparameters
print(search.best_params_)


{'learning_rate': 0.25, 'max_depth': 10, 'n_estimators': 200}


In [153]:
xg_std = xgb.XGBClassifier(**search.best_params_)  #{'learning_rate': 0.3, 'max_depth': 8, 'n_estimators': 200}
#{'learning_rate': 0.2, 'max_depth': 12, 'n_estimators': 200}
xg_std.fit(xtrain_std_som,xg_ytrain_smo)

y_pred_std = xg_std.predict(xtest_std)

print(classification_report(y_pred_std,xg_ytest))

              precision    recall  f1-score   support

           0       0.26      0.40      0.32       124
           1       0.96      0.93      0.95      2000

    accuracy                           0.90      2124
   macro avg       0.61      0.67      0.63      2124
weighted avg       0.92      0.90      0.91      2124



In [154]:
xg_std = xgb.XGBClassifier(**{'learning_rate': .3, 'max_depth': 8, 'n_estimators': 200})

xg_std.fit(xtrain,xg_ytrain)

y_pred_std = xg_std.predict(xtest)

print(classification_report(y_pred_std,xg_ytest))

              precision    recall  f1-score   support

           0       0.21      0.48      0.30        86
           1       0.98      0.93      0.95      2038

    accuracy                           0.91      2124
   macro avg       0.60      0.70      0.62      2124
weighted avg       0.95      0.91      0.92      2124



In [155]:
xg_std = xgb.XGBClassifier(**{'learning_rate': 0.055, 'max_depth': 10, 'n_estimators': 200})

xg_std.fit(xtrain_std_som,xg_ytrain_smo)

y_pred_std = xg_std.predict(xtest_std)

print(classification_report(y_pred_std,xg_ytest))

              precision    recall  f1-score   support

           0       0.34      0.40      0.37       159
           1       0.95      0.94      0.94      1965

    accuracy                           0.90      2124
   macro avg       0.64      0.67      0.65      2124
weighted avg       0.90      0.90      0.90      2124



In [156]:
xg_std = xgb.XGBClassifier(**{'learning_rate': 0.0007, 'max_depth': 8, 'n_estimators': 300})

xg_std.fit(xtrain_std_som,xg_ytrain_smo)

y_pred_std = xg_std.predict(xtest_std)

print(classification_report(y_pred_std,xg_ytest))

              precision    recall  f1-score   support

           0       0.38      0.29      0.33       255
           1       0.91      0.94      0.92      1869

    accuracy                           0.86      2124
   macro avg       0.64      0.61      0.62      2124
weighted avg       0.84      0.86      0.85      2124



In [157]:
#{'learning_rate': 0.007, 'max_depth': 8, 'n_estimators': 300} 0.39 0.93 0.88
#{'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 300} 0.38 0.93 0.88  
#{'learning_rate': 0.0019, 'max_depth': 5, 'n_estimators': 300} 0.42 0.92 0.87
#{'learning_rate': 0.0019, 'max_depth': 5, 'n_estimators': 200} 0.39 0.92 0.87
#{'learning_rate': 0.019, 'max_depth': 5, 'n_estimators': 250} 0.49 0.92 0.88

In [158]:
xg_std = xgb.XGBClassifier(**{'learning_rate': 0.019, 'max_depth': 5, 'n_estimators': 250})

xg_std.fit(xtrain_std_som,xg_ytrain_smo)

y_pred_std = xg_std.predict(xtest_std)

print(classification_report(y_pred_std,xg_ytest))

              precision    recall  f1-score   support

           0       0.48      0.36      0.41       252
           1       0.92      0.95      0.93      1872

    accuracy                           0.88      2124
   macro avg       0.70      0.65      0.67      2124
weighted avg       0.86      0.88      0.87      2124



In [159]:
xg_std.predict_proba(xtest_std)

array([[0.58159256, 0.41840747],
       [0.43218583, 0.5678142 ],
       [0.3794288 , 0.6205712 ],
       ...,
       [0.27257365, 0.72742635],
       [0.00675344, 0.99324656],
       [0.30483872, 0.6951613 ]], dtype=float32)

In [160]:
xg_std.predict(xtest_std)

array([0, 1, 1, ..., 1, 1, 1])

## Pipeline

In [161]:
from  sklearn.model_selection import train_test_split

xg_y = y.replace({'full_trip' : 1,'voided':0})

xtrain, xtest, ytrain,ytest=train_test_split(x, xg_y , test_size=0.2, random_state=42, stratify=y)

In [162]:
from imblearn.pipeline import Pipeline

In [163]:
pipe = Pipeline(
    steps=[
        ('encoder',BinaryEncoder(drop_invariant=True)),
        ('scaler',MinMaxScaler()),
        ('sampler',RandomOverSampler(random_state=52,sampling_strategy=.5)),
        ('model',xgb.XGBClassifier(**{'learning_rate': 0.019, 'max_depth': 5, 'n_estimators': 200}))]
)


pipe.fit(xtrain,ytrain)
y_pred = pipe.predict(xtest)

print(classification_report(y_pred,ytest))

              precision    recall  f1-score   support

           0       0.48      0.37      0.42       246
           1       0.92      0.95      0.93      1878

    accuracy                           0.88      2124
   macro avg       0.70      0.66      0.67      2124
weighted avg       0.87      0.88      0.87      2124



In [164]:
pipe.predict_proba(xtest)

array([[0.57753885, 0.42246112],
       [0.48908103, 0.510919  ],
       [0.36652583, 0.6334742 ],
       ...,
       [0.28032917, 0.71967083],
       [0.01369303, 0.98630697],
       [0.28957278, 0.7104272 ]], dtype=float32)

In [165]:
xtrain

,paid at,accepts marketing,billing province,shipping province,payment method,source,created month,created day,shipping,total,lineitem quantity
9670,0,no,C,C,Cash on Delivery (COD),shopify_draft_order,9,11,35.0,156.50,2
6829,0,no,IS,IS,Cash on Delivery (COD),web,4,4,50.0,468.50,3
408,1,no,GZ,GZ,Cash on Delivery (COD),web,2,3,32.0,540.55,5
6136,1,no,C,C,others,web,2,6,35.0,2065.30,15
47,1,no,C,C,Cash on Delivery (COD),web,1,28,32.0,314.00,5
...,...,...,...,...,...,...,...,...,...,...,...
6093,0,no,BH,BH,Cash on Delivery (COD),web,2,5,40.0,713.60,12
9608,0,no,ALX,ALX,others,shopify_draft_order,8,30,40.0,827.50,7
475,1,no,C,C,Cash on Delivery (COD),web,2,4,32.0,173.50,3
1153,1,no,KB,KB,Cash on Delivery (COD),web,7,3,32.0,463.50,8


In [166]:
test= {'paid at': [0,1,0,0,1] ,
         'accepts marketing':['no','no','yes','no','yes'],
         'billing province':['ALX','out of egypt','C','IS','C'],
         'shipping province':['ALX','out of egypt','C','IS','C'],
         'payment method':['Cash on Delivery (COD)','others','Cash on Delivery (COD)','Cash on Delivery (COD)','Cash on Delivery (COD)'],
         'source':['web','shopify_draft_order','web','shopify_draft_order','web'],
         'created month':[2,7,8,9,4],
         'created day':[5,2,7,11,4],
         'shipping':[60,3000,50,35,50],
         'total':[267.85,5511.5,1542,150,500],
         'lineitem quantity':[10,15,20,2,4]}

In [167]:
pipe.predict_proba(pd.DataFrame(test))

array([[0.23324662, 0.7667534 ],
       [0.0840705 , 0.9159295 ],
       [0.28848124, 0.71151876],
       [0.25484788, 0.7451521 ],
       [0.01570743, 0.98429257]], dtype=float32)

## Saving Model

In [168]:
import pickle 

In [169]:
pickle.dump(pipe,open('Full_Trip_Prediction.pkl','wb'))

In [170]:
description_df = pd.read_excel('Full_Trip_Prediction_Description.xlsx')
description_df

,Column_Num,Name,Uniques,How to get
0,1,paid at,"[0,1]",From paid at feature if there is date [1] then...
1,2,accepts marketing,"['no', 'yes']",Based on Shopify
2,3,billing province,"['C', 'IS', 'GZ', 'DK', 'ALX', 'SU', 'KB', 'BH...",For billing province feature if it like one of...
3,4,shipping province,"['C', 'IS', 'GZ', 'DK', 'ALX', 'SU', 'KB', 'BH...",For shipping province feature if it like one o...
4,5,payment method,"['Cash on Delivery (COD)', 'others']",Payment method if not [Cash on Delivery (COD)]...
5,6,source,"['shopify_draft_order', 'web']",Source if not [web] then [shopify_draft_order]...
6,7,created month,Based on Shopify,From created at feature get month
7,8,created day,Based on Shopify,From created at feature get day
8,9,shipping,Based on Shopify,Based on Shopify
9,10,total,Based on Shopify,Based on Shopify
